# 中間画像生成

In [7]:
import numpy
import cv2
import os
import glob
import pandas as pd
import shutil

def mid_image(path1, path2, save_dir_name):
    #print(path1)
    imgStart = cv2.imread(path1)
    imgEnd = cv2.imread(path2)
    #ファイル名生成用
    img_name = os.path.splitext(os.path.basename(path1))
    #ディレクトリ
    #dir_name = os.path.dirname(path1)
    # 0.0～1.0
    imgStart = (imgStart / 255)
    imgEnd = (imgEnd / 255)
    #書き込み
    imgTmp = (imgStart / 2  + imgEnd / 2)
    imgRes = cv2.resize(imgTmp,(640, 480))
    save_name = save_dir_name+ img_name[0] +"_mid_"+ img_name[1]
    
    #cv2.imshow("color", imgRes)
    cv2.imwrite(save_name, imgTmp * 255)
    cv2.waitKey(10)

    #cv2.waitKey(0)
    cv2.destroyAllWindows()
    return save_name
    

# メイン処理

In [8]:

#csvリスト
path_get_dir = r"C:\Users\Ten\python\work\30_60transac\aaa"
csv_list = []
for curDir, dirs, files in os.walk(path_get_dir):
    for file in files:
        if file.endswith(".csv"):
            csv_list.append(os.path.join(curDir, file))

print(csv_list)           

#csvごとに更新
for csv in csv_list:
    #新しく作成したimgのリスト
    img_new_name_list = []
    dir_name_csv = os.path.dirname(os.path.dirname(csv))
    gt_path = dir_name_csv+"\\gt\\"
    img_path = dir_name_csv+"\\img\\"
    gt_void_path = dir_name_csv+"\\gt\\Void\\"
    
    #処理①imgフォルダを複数にする
    #元画像ディレクトリパス
    
    img_files = glob.glob(img_path+"*.jpg")
    for i in range(0,len(img_files)):
        try:
            i_file1 = img_files[i]
            i_file2 = img_files[i+1]
            name = mid_image(i_file1,i_file2,img_path)
            #listに格納
            img_new_name_list.append(name)
        except:
            pass
    
    #処理②voidフォルダを複数にする
    void_files = glob.glob(gt_void_path+"*.jpg")
    for j in range(0,len(void_files)):
        try:
            v_file1 = void_files[j]
            v_file2 = void_files[j+1]
            mid_image(v_file1,v_file2,gt_void_path)
        except:
            pass    
    
    #処理③gtのcsvファイルを増やす
    
    #gt読み込み
    gt_read = pd.read_csv(csv)
    gt_read["temp"] = gt_read.index

    #中間生成対象行
    mid = ["H","W"]

    #flag行
    flag = ["occ", "frame"]

    #中間生成
    gt_temp = gt_read[mid]-gt_read[mid].diff(periods=-1).applymap(lambda x:x/2)
    gt_temp[mid] = gt_temp[mid].applymap('{:.2f}'.format)
    #flagはひとつ前をコピーする
    gt_temp[flag] = gt_read[flag]
    #中間に挿入するための行
    gt_temp["temp"] = gt_temp.index+0.5
    #ファイル名追加
    gt_temp["path"] = pd.DataFrame(img_new_name_list)
    

    #結合
    gt = pd.concat([gt_read,gt_temp]).sort_values("temp")
    gt = gt.reset_index(drop=True)
    gt = gt.drop("temp", axis=1)
    gt = gt.dropna()
    gt.to_csv(csv, index=False)
    
    #print(dir_name_csv)
    



['C:\\Users\\Ten\\python\\work\\30_60transac\\aaa\\1\\gt\\1.csv', 'C:\\Users\\Ten\\python\\work\\30_60transac\\aaa\\2\\gt\\1.csv']
